In [ ]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/stsb-roberta-large', device='cuda')
scores = model.predict([('Sentence 1', 'Sentence 2'), ('Sentence 3', 'Sentence 4')])

In [1]:
from lm_polygraph.estimators import SAR
from lm_polygraph import estimate_uncertainty
from lm_polygraph.utils.model import WhiteboxModel
from transformers import logging, DebertaForSequenceClassification, DebertaTokenizer
import torch
from transformers import AutoModelForCausalLM, LlamaForCausalLM, AutoTokenizer, AutoConfig

# %%
model_path = 'mistralai/Mistral-7B-Instruct-v0.3'
model = AutoModelForCausalLM.from_pretrained(model_path, device_map='auto', torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.config.pad_token_id = 2
model.generation_config.pad_token_id = 2
tokenizer.pad_token = tokenizer.eos_token

# %%
print(tokenizer.is_fast)

# %%
question = 'Answer the question in five words or less: Who is the president of the United States?'
ue_model = WhiteboxModel(model, tokenizer)
output = estimate_uncertainty(ue_model, SAR(), question)

/home/hanwenli/anaconda3/envs/debug/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.81s/it]


True


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


In [4]:
prompt_template = """You are a chemistry assistant that helps identify the correct tool to call for molecular property analysis and extract the SMILES string from a user's question.

AVAILABLE_TOOLS:
- CalculateMolecularWeight: Use this tool when you need to calculate the molecular weight of a SMILES string. Units in Dalton.
- CalculateQED: Compute Quantitative Estimate of Druglikeness (QED) of the given molecule.
- BrenkFilter: Used when you need to calculate whether a molecule triggers the Brenk Filter.
- CalculateTPSA: Compute the Topological polar surface area (TPSA) of the given molecule.
- CalculateBBBPermeant: Calculates the Blood Brain Barrier Permeability of the compound.
- calculate_druglikeness: Calculates the druglikeness of the compound with regards to Lipinski's rule of 5.
- calculate_gi_absorption: Returns whether the gastrointestinal absorption is high or low.
- CalculateLogP: Use this tool when you need to calculate the log of the partition coefficient (LogP) of a compound.
- PainsFilter: Used when you need to calculate whether a molecule triggers the PAINS Filter.
- CalculateSA: Used to compute the synthetic accessibility (SA) of the given molecule.

Your task:
1. Extract the SMILES string from the question.
2. Determine the correct tool name to call based on the intent.

Respond in the following format:
SMILES: <smiles_string>  
Function: <tool_name>

---

Example 1:  
Question: What is the QED of CCO?  
SMILES: CCO  
Function: CalculateQED

Example 2:  
Question: Is the SMILES: C=C1C(C2(C(C(C1(C2(Cl)Cl)Cl)Cl)Cl)Cl)(CCl)CCl, druglike?  
SMILES: C=C1C(C2(C(C(C1(C2(Cl)Cl)Cl)Cl)Cl)Cl)(CCl)CCl  
Function: calculate_druglikeness

---

Question: {input}
"""
question = prompt_template.format(input='hi')
print(question)

You are a chemistry assistant that helps identify the correct tool to call for molecular property analysis and extract the SMILES string from a user's question.

AVAILABLE_TOOLS:
- CalculateMolecularWeight: Use this tool when you need to calculate the molecular weight of a SMILES string. Units in Dalton.
- CalculateQED: Compute Quantitative Estimate of Druglikeness (QED) of the given molecule.
- BrenkFilter: Used when you need to calculate whether a molecule triggers the Brenk Filter.
- CalculateTPSA: Compute the Topological polar surface area (TPSA) of the given molecule.
- CalculateBBBPermeant: Calculates the Blood Brain Barrier Permeability of the compound.
- calculate_druglikeness: Calculates the druglikeness of the compound with regards to Lipinski's rule of 5.
- calculate_gi_absorption: Returns whether the gastrointestinal absorption is high or low.
- CalculateLogP: Use this tool when you need to calculate the log of the partition coefficient (LogP) of a compound.
- PainsFilter: 

In [2]:
output

UncertaintyOutput(uncertainty=-8.546707266514412, input_text='Answer the question in five words or less: Who is the president of the United States?', generation_text='Joe Biden (as of March 2023)', generation_tokens=[8601, 22145, 1093, 1061, 1070, 4885, 29473, 29518, 29502, 29518, 29538, 29499], model_path=None, estimator='SAR')